In [2]:
import numpy as np
import random
import math
import hashlib as hash

class Group:       
    def eval(self, g1,g2):
        pass
    def getOrder(self):
        pass
    def getInverse(self, g):
        pass 



class Dihedral(Group):
    
     def __init__(self, n):
        self.n=n
        self.__createTable()
        

     def __createTable(self): 
          n=self.n
          #print(n)
          table=[]
          row1=[i for i in range(n)] #0 to n-1
          row2=[i for i in range(n,2*n)] #n to 2n-1
          table.append(row1+row2) # 0, 2, 4, 6,.. 2n-2
          for i in range(1,n):
              row1=row1[1:]+row1[:1] #(1,..n-1)+(0,..n-2)=(1,3, ..., 2n-3)
              row2=row2[1:]+row2[:1] #(n+1,n+2,.., 2n-1)+(n, n+1,.., 2n-2)=(2n+1,2n+3...,4n-3)
              table.append(row1+row2)#(2n+2,2n+6,..6n-6)

          row1=[n]
          row2=[0]
          for j in range(n-1,0,-1):
               row1.append(n+j)
               row2.append(0+j)
          table.append(row1+row2)

          for i in range(1,n):
               row1 = (row1[-1:] + row1[:-1])
               row2 = (row2[-1:] + row2[:-1])
               table.append(row1+row2)
                
          self.table=table        
            
     
     def eval(self, g1,g2):
         return self.table[g1][g2] # input numbers g1, g2
    
     def getOrder(self):
         return 2*self.n 

     def getInverse(self, g):    
         for i in range(2*self.n):
            if (0==self.table[g][i]):
               return i   


  
class FP:

   def __init__(self,rep, p):
          self.p=p
          self.rep=rep

   def __add__(self, b):
          rep=(self.rep + b.rep)%self.p
          return FP(rep,self.p)

   def __sub__(self, b):
          rep=(self.rep - b.rep)%self.p
          return FP(rep,self.p)

   def getRandomElement(self):
           rep=int(random.randint(0,self.p-1))
           
           return FP(rep,self.p)

   def getZero(self):
           return FP(0,self.p)

   def getOne(self):
           return FP(1,self.p)

   def getMinusOne(self):
           return FP(self.p-1,self.p)
        
   def getRandomNonZeroElement(self):
           element=random.randint(1,self.p-1)
           return FP(element,self.p)

   def __pow__(self, n):  
          return FP(pow(self.rep,n,self.p), self.p)
   
   def getSquare(self):
        g=self.getRandomNonZeroElement()
        one=self.getOne()
        output=g**((self.p-1)//2)
        while not output.equals(one):
            g=self.getRandomNonZeroElement()
            output=g**((self.p-1)//2)

        return g

   def getNonSquare(self):
        g=self.getRandomNonZeroElement()
        one=self.getOne()
        output=g**((self.p-1)//2)
        while  output.equals(one):
            g=self.getRandomNonZeroElement()
            output=g**((self.p-1)//2)
        return g
   def getOrder(self):
          return self.p    
   def getPrimitiveRoot(self):
          
          if self.p==17:
             g=self.getRandomNonZeroElement()
            
             g8=g**8
             
             while g8.rep ==1 : 
               g=self.getRandomNonZeroElement()
               g8=g**8
             gi=g**15
          
             return g,gi

   def __mul__(self, b):
          rep=(self.rep*b.rep)%self.p
          
          return FP(rep,self.p)
      
   def equals(self,a):
          return self.rep==a.rep

   def __str__(self):        
         return str(self.rep)

   def isOne(self):
           return self.rep==1

   def isNonZero(self):
           return self.rep!=0

          

class  TwoCocycleLambda:
     
     def __init__(self,dihedral, field, square):
         self.group = dihedral
         self.field=field
         if square:
            self.lamb=self.field.getSquare() # a random square
         else:
            self.lamb=self.field.getNonSquare() # a random nonsquare

         self.__generator()
         
          
     def __generator(self):
         order=self.group.getOrder()
         n=order//2
         rep=[]
         one=self.field.getOne()
         
         
         for i in range(0,order*order,1):
            rep.append(one)
         for i in range(n,order): # order is just 2n
            for j in range(n,order):
                  rep[i+order*j]=self.lamb #rep[n+2n*n]=lambda
                    
         self.representation = rep 
     
     def isInvolution(self): # it is an involution if and only if g*h evaluated for all pairs (g,h) is 1
         order=self.group.getOrder() # returns 2n
         for g in range(order):
            for h in range(order):
                o2=self.evaluate(g,h)*self.evaluate(g,h)
                if not o2.isOne():
                   return False
         return True
                   
                
         
     def is2cocycle(self):
         order=self.group.getOrder()
         one=self.field.getOne()
         for i in range(order):
            o1=self.evaluate(i,0)
            o2=self.evaluate(0,i)
            #print(i,o1,o2)
            if not o1.equals(o2) or not o2.equals(one):
                return False
            j=self.group.getInverse(i)
            #print(i,j)
            bit1=self.evaluate(i,j)
            bit2=self.evaluate(j,i)
            if not bit2.equals(bit1):
              return False
         
         
        
         for g in range(order):
            for h in range(order):
              for k in range(order):
                  
                  gh=self.group.eval(g,h)
                  o1=self.evaluate(g,h)*self.evaluate(gh,k)
                  hk=self.group.eval(h,k)
                  o2=self.evaluate(g,hk)*self.evaluate(h,k)
                  
                  if not o2.equals(o1):
                     print((g,h),self.evaluate(g,h))
                     print((gh,k),self.evaluate(gh,k))
                     print((g,hk),self.evaluate(g,hk))
                     print((h,k),self.evaluate(h,k))
                     print(o1,o2)
                     return False
       
         return True


     def  evaluate(self,g1,g2): # g1, g2 are numbers,
         order=self.group.getOrder() # 2n
        
         n=g1+order*g2
         return self.representation[n]



class  TwoCocycleDnK:
     
     def __init__(self,group, primtiveRoot,primtiveRootI):
         self.group = group
         self.primitive=primtiveRoot
         self.primtiveRootI=primtiveRootI
         self.__generator(primtiveRoot)
         
     
     def __generator(self, t):
         order=self.group.getOrder() #2n
         rep=[]
         m=int(order/2)

         for i in range(0,order*order,1):
            rep.append(self.primitive**0) # just 1s

         for i in range(m, order,1):
            for j in range(0,order,1):
                #print(i,j)
                n1=i+order*j
                exp=j%m
                #print(exp)
                rep[n1]= self.primitive**exp # same definition as two cycle lambda
        
         self.representation = rep 
     
     def isInvolution(self):
         order=self.group.getOrder()
         for g in range(order):
            for h in range(order):
                o2=self.evaluate(g,h)*self.evaluate(g,h)
                if not o2.isOne():
                   return False
         return True
                   
                
         
     def is2cocycle(self):
         order=self.group.getOrder()
         one=self.primitive.getOne()
         for i in range(order):
            o1=self.evaluate(i,0)
            o2=self.evaluate(0,i)
            #print(i,o1,o2)
            if not o1.equals(o2) or not o2.equals(one):
                return False
            j=self.group.getInverse(i)
            #print(i,j)
            bit1=self.evaluate(i,j)
            bit2=self.evaluate(j,i)
            if not bit2.equals(bit1):
              return False
         
         
        
         for g in range(order):
            for h in range(order):
              for k in range(order):
                  
                  gh=self.group.eval(g,h)
                  o1=self.evaluate(g,h)*self.evaluate(gh,k)
                  hk=self.group.eval(h,k)
                  o2=self.evaluate(g,hk)*self.evaluate(h,k)
                  
                  if not o2.equals(o1):
                     print((g,h),(gh,k))
                     print((g,hk),(h,k))
                     print(o1,o2)
                     return False
       
         return True


     def  evaluate(self,g1,g2):
         order=self.group.getOrder()
        
         n=g1+order*g2
         return self.representation[n]
         


class algebraKD2nElement:
      

      def __init__(self, group, cocycle, rep):
         self.group=group
         self.rep=rep
         self.cocycle=cocycle
      
      def __add__(self, element):
         order=self.group.getOrder()
         L=[None]*order
         for i in range(order):
            L[i]=(self.rep[i]+element.rep[i])
               
         return algebraKD2nElement(self.group,self.cocycle,L)

      def getAstElement(self,ti):
          order=self.group.getOrder()
          L=[None]*order
          m=int(order/2)
          for i in range(m):
             L[i]=self.rep[i]*(ti**i)
             L[i+m]=self.rep[i+m]*(ti**i) 
             

          return algebraKD2nElement(self.group, self.cocycle,L)

      def __sub__(self,element):
         order=self.group.getOrder()
         L=[None]*order
         for i in range(order):
            L[i]=(self.rep[i]-element.rep[i])
         return algebraKD2nElement(self.group,self.cocycle,L)
      
      def getNumber(self):
         #((0+a2)p+a1)p+a0
         orderF=self.rep[0].getOrder()
         n=len(self.rep)-1
         number=0
         for i in range(n,0,-1):
            number=(number+self.rep[i].rep)*orderF
            
         number=number+self.rep[0].rep
         return number
      def getAdjunct(self):
          order=self.group.getOrder()
          L=[None]*order
          for i in range(len(self.rep)):
              j=self.group.getInverse(i)
              coef=self.rep[i]*self.cocycle.evaluate(i,j )
              L[j]=coef
          return algebraKD2nElement(self.group,self.cocycle,L)    

      def __mul__(self, element):
         order=self.group.getOrder()
         
         L=[None]*order
         for i in range(order):
            L[i]=self.rep[0].getZero()


         for i in range(order):
            for j in range(order):
                index=self.group.eval( i,j)
                L[index]=(L[index]+self.rep[i]*element.rep[j]*cocycle.evaluate(i,j))

         return algebraKD2nElement(self.group,self.cocycle,L)

      def equals(self,b):
          order=self.group.getOrder()
          for i in range(order):
            if not self.rep[i].equals(b.rep[i]):
               return False

          return True

      def __str__(self):
          stri="["
          for i in range(len(self.rep)-1):
             stri=stri+self.rep[i].__str__()+","

          stri=stri+self.rep[len(self.rep)-1].__str__()+"]"
          return stri
      
     

class algebraKD2n:
      @staticmethod
      def getPublicElement( group, cocycle,K):
          pr=False
          order=group.getOrder()
          n=order//2
          while not pr:
            
            element=algebraKD2n.getRandomElement( group, cocycle,K)
            i=0
            pr1=False
            while i<n and not pr1:
              if element.rep[i].isNonZero():
                 pr1=True
              i=i+1
            i=n
            pr2=False
            while i<order and not pr2:
              if element.rep[i].isNonZero():
                 pr2=True
              i=i+1
            pr= pr1 and pr2
          return element

      @staticmethod
      def getRandomElement( group, cocycle,K):
        order=group.getOrder()
        L=[None]*order
        for i in range(order):
           L[i]=K.getRandomElement()
        
        return algebraKD2nElement(group, cocycle,L)
      @staticmethod
      def getCnElement(group,cocycle,K):
          order=group.getOrder()
          L=[None]*order
          n=order//2    
          for i in range(order):
             if i<n:
               L[i]=K.getRandomElement()
             else:
               L[i]=K.getZero()
      
          return algebraKD2nElement(group, cocycle,L)

      @staticmethod
      def getTElement(group, cocycle,K):
          order=group.getOrder()
          L=[None]*order
          n=order//2
          for i in range(0,n):
             L[i]=K.getZero()
          n1=n//2
          L[n]=K.getRandomElement()
          for i in range(1,n1+1): 
              L[i+n]=K.getRandomElement()
              L[n+(n-i)%n]=L[i+n]  

          return algebraKD2nElement(group, cocycle,L)
      @staticmethod
      def getCnyElement(group, cocycle,K):
          order=group.getOrder()
          L=[None]*order
          n=order//2
          for i in range(0,n):
              L[i]=K.getZero()
                
          for i in range(n,order):
              L[i]=K.getRandomElement()
                 
          return algebraKD2nElement(group, cocycle,L)
        
        
def calc_bd(b,d): # given vectors b, d of length m over F_p, calculates [bd] whose lth component is b_0d_l+b_1d_{l-1}+...b_{m-1}d_{l+1}
    m=len(b)
    a=[FP(1,p).getZero()]*m
    for l in range(0,m):
        S=FP(1,p).getZero()
        for i in range(0,m):
            j=(l+m-i)%m
            s=((b[i]).__mul__(d[j]))
            S=S.__add__(s)
        a[l]=S
    return a      

def vector_to_element_xy(b): 
    order=group.getOrder()
    L=[None]*order
    m=int(order/2)
    for i in range(m):
        L[i]=FP(1,p).getZero()
        L[i+m]=b[i]
    return algebraKD2nElement(group, cocycle,L)


def element_to_vector_xy(elt): 
    order=group.getOrder()
    elt=elt.rep
    m=int(order/2)
    L=[None]*m
    for i in range(m):
        L[i]=elt[i+m]
    return L

def vector_to_element_x(b): 
    order=group.getOrder()
    L=[None]*order
    m=int(order/2)
    for i in range(m):
        L[i]=b[i]
        L[i+m]=FP(1,p).getZero()
    return algebraKD2nElement(group, cocycle,L)


def element_to_vector_x(elt): 
    order=group.getOrder()
    elt=elt.rep
    m=int(order/2)
    L=[None]*m
    for i in range(m):
        L[i]=elt[i]
    return L

def y_to_vw(elt): 
    order=group.getOrder()
    elt=elt.rep
    m=int(order/2)
    w=[None]*m
    v=[None]*m
    for i in range(m):
        v[i]=elt[i]
        w[i]=elt[i+m]
    return v,w

def construct_circulant_b(b):
    vec=b.copy()
    n=len(vec)
    shifted_vecs=[]
    for col in range(0,n):
        si=(n-col)%n
        shifted=[]
        for j in range(0, n):
            ind=(si+j)%n
            shifted.append(vec[ind])
        shifted_vecs.append(shifted)
    M=np.array(shifted_vecs).reshape(n,n)
    M=np.transpose(M)
    return M

def construct_circulant_bd(b,d):
    vec=calc_bd(b,d)
    n=len(vec)
    shifted_vec=[]
    for col in range(0,n):
        si=(n-col)%n
        for j in range(0, n):
            ind=(si+j)%n
            shifted_vec.append(vec[ind])
    M=np.array(shifted_vec).reshape(n,n)
    M=np.transpose(M)
    return M

def to_GF_array(bd):
    bd_str=bd
    for i in range(0, n):
        for j in range(0,n):
            bd_str[i][j]=int(bd[i][j].__str__())
    M = Matrix(GF(p), bd_str)
    return M
    
def to_GF_vec(b):
    n=len(b)
    b_str=b
    for i in range(0, n):
        b_str[i]=int(b[i].__str__())
    M = Matrix(GF(p), b_str)
    return b_str

def to_FP_vec(sol):
    n=len(sol)
    sol_str=[None]*n
    for i in range(0, n):
        elt=int(sol[i])
        sol_str[i]=FP(elt,p)
    return sol_str

In [13]:
n=41
p=41
m=1
group=Dihedral(n)
field=FP(1,p)
cocycle=TwoCocycleLambda(group,field,False)
#cocycle=TwoCocycleDnK(group,field,False)
lam=cocycle.representation[len(cocycle.representation)-1]
lami=(lam.__pow__(-1)).rep
lami=int(lami)
print("lambda:", lam)
print("Public parameter h")
h=algebraKD2n.getPublicElement( group, cocycle,field)
c,d=y_to_vw(h)
a1=algebraKD2n.getCnElement(group,cocycle,field)
t1=algebraKD2n.getTElement(group,cocycle,field)
pk1=a1*h*t1
v,w=y_to_vw(pk1)
b=[None]*n  # b is fixed randomly
for i in range(0,ceil(n/2)):
    b[i]= FP(1,p).getRandomElement()
for i in range(ceil(n/2),n):
    b[i]= b[n-i]
#b_dash=btobdash(b,lam)
bd=construct_circulant_bd(b,d)
bc=construct_circulant_bd(b,c)
bds=to_GF_array(bd)
bcs=to_GF_array(bc)
bs=to_GF_vec(b)
cs=to_GF_vec(c)
ds=to_GF_vec(d)
vs=to_GF_vec(v)
ws=to_GF_vec(w)
sol1=bcs.solve_right(ws)
sol2=bds.solve_right(vs)
sol22=sol2*lami
print("two solutions are same:", sol1==sol22)
a=sol1
print("sol1 is correct:", list(bcs*sol1)==list(ws))
print("sol2 is correct:",list(bds*sol2)==list(vs))
print("bds and bcs are invertible", bds.is_invertible(), bcs.is_invertible())
bdsi=bds.inverse()
bcsi=bcs.inverse()
print("inverse equations hold:", bdsi*vector(vs)==sol2,bcsi*vector(ws)==sol1)
a=to_FP_vec(sol1)
B=to_FP_vec(b)
a0=vector_to_element_x(a)
B0=vector_to_element_xy(B)
print("a0*h*B0 retrives the public key:", (a0*h*B0).equals((pk1)))
print("a1*h*t1 retrives the public key:", (a1*h*t1).equals((pk1)))
avec=vector_to_element_x(a)
print("public parameter h:", h)
print("original a recovered:",a0.equals(a1))
print("original b recovered:",B0.equals(t1))
print("original a:", a1,"original t:", t1, "computed a:", a0, "computed t:", B0)

lambda: 29
Public parameter h
two solutions are same: True
sol1 is correct: True
sol2 is correct: True
bds and bcs are invertible True True
inverse equations hold: True True
a0*h*B0 retrives the public key: True
a1*h*t1 retrives the public key: True
original a recovered: False
original b recovered: False
original a: [24,2,12,32,10,2,27,1,5,7,17,32,7,24,28,26,17,8,32,18,13,8,19,17,0,11,33,17,27,1,36,3,33,9,30,34,22,26,21,5,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] original t: [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,8,18,11,31,9,34,3,16,39,32,0,15,31,3,26,0,31,39,4,40,40,4,39,31,0,26,3,31,15,0,32,39,16,3,34,9,31,11,18,8] computed a: [39,9,4,23,8,8,10,40,31,27,22,36,11,14,35,28,25,0,0,10,16,33,24,6,33,17,15,13,17,10,18,31,33,16,13,28,2,36,37,13,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] computed t: [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [ ]:
# Computing the probability of M_c being invertible
n=23
p=23
group=Dihedral(n)
field=FP(1,p)
cocycle=TwoCocycleLambda(group,field,False)
prob=0
trials=10000
for i in range(0,trials):
    h=algebraKD2n.getPublicElement( group, cocycle,field)
    c,d=y_to_vw(h)
    c_circ=construct_circulant_b(c)
    c_circ=to_GF_array(c_circ)
    if c_circ.is_invertible():
        prob=prob+1/trials
round(prob+1/p-1, 3) # proximity to the computed value, 1-1/p, up to 3 decimal places. 

# 